In [1]:
import json
from pathlib import Path

def load_all_results(logs_dir='logs'):
    results = []
    logs_path = Path(logs_dir)
    
    # Check if logs directory exists
    if not logs_path.exists():
        print(f"Warning: Directory '{logs_dir}' not found.")
        return results
    
    # Find all result.json files recursively
    result_files = list(logs_path.rglob('result.json'))
    
    if not result_files:
        print(f"No result.json files found in {logs_dir}")
        return results
    
    # Load each result file
    for result_file in result_files:
        try:
            with open(result_file, 'r') as f:
                data = json.load(f)
                results.append(data)
                print(f"Loaded: {result_file}")
        except json.JSONDecodeError:
            print(f"Error: Could not parse JSON from {result_file}")
        except Exception as e:
            print(f"Error reading {result_file}: {str(e)}")
    
    print(f"\nSuccessfully loaded {len(results)} result files.")
    return results

all_results = load_all_results()


Loaded: logs/2025-07-09_21-53-00_Randy_openai_o4-mini_game/result.json
Loaded: logs/2025-07-09_20-46-24_openai_o4-mini_Randy_game/result.json
Loaded: logs/2025-07-06_07-55-42_openai_o3_Alice_game/result.json
Loaded: logs/2025-07-11_10-19-41_Randy_anthropic_claude-4-sonnet-20250522_game/result.json
Loaded: logs/2025-07-09_21-08-02_anthropic_claude-4-sonnet-20250522_google_gemini-2.5-pro_game/result.json
Loaded: logs/2025-07-11_10-19-41_Randy_openai_o3_game/result.json
Loaded: logs/2025-07-09_20-46-24_Randy_anthropic_claude-4-sonnet-20250522_game/result.json
Loaded: logs/2025-07-11_10-19-41_Randy_google_gemini-2.5-pro_game/result.json
Loaded: logs/2025-07-11_09-50-48_anthropic_claude-4-sonnet-20250522_Randy_game/result.json
Loaded: logs/2025-07-06_08-32-19_Alice_google_gemini-2.5-pro_game/result.json
Loaded: logs/2025-07-11_10-26-56_google_gemini-2.5-pro_anthropic_claude-4-sonnet-20250522_game/result.json
Loaded: logs/2025-07-07_20-32-58_Randy_openai_o4-mini_game/result.json
Loaded: logs

In [2]:
def merge_random_players(result):
    players = []
    for p in result["players"]:
        if p['name'] == 'Alice':
            players.append('Randy')
        else:
            players.append(p['name'])
      
    winner  = result["winner"]
    if winner == 'Alice':
        winner = 'Randy'
    
    return players, winner


In [3]:
import pandas as pd
from collections import defaultdict
from IPython.display import display

# --- hyper-params ------------------------------------------------------------
K          = 32          # Elo K-factor
START_ELO  = 1000        # initial rating
# -----------------------------------------------------------------------------


# containers for running stats
ratings = defaultdict(lambda: START_ELO)
stats   = defaultdict(lambda: {"wins": 0, "losses": 0, "games": 0})

for result in all_results:                       # iterate every match record
    players, winner = merge_random_players(result)
    loser = next(p for p in players if p != winner)

    # ---- win / loss counts --------------------------------------------------
    stats[winner]["wins"]  += 1
    stats[loser]["losses"] += 1
    stats[winner]["games"] += 1
    stats[loser]["games"]  += 1

    # ---- Elo update ---------------------------------------------------------
    Ra, Rb           = ratings[winner], ratings[loser]
    Ea               = 1 / (1 + 10 ** ((Rb - Ra) / 400))
    ratings[winner] += K * (1 - Ea)          # winner’s new rating
    ratings[loser]  += K * (0 - (1 - Ea))    # loser’s new rating (Eb = 1-Ea)

# ---- assemble DataFrame -----------------------------------------------------
rows = []
for player, s in stats.items():
    wins, losses, games = s["wins"], s["losses"], s["games"]
    rows.append(
        {
            "player"   : player,
            "wins"     : wins,
            "losses"   : losses,
            "games"    : games,
            "win_rate" : wins / games,
            "elo"      : round(ratings[player], 2),
        }
    )

df = (
    pd.DataFrame(rows)
      .sort_values("elo", ascending=False)
      .reset_index(drop=True)
)

# nice display 🪄
display(
    df.style
      .format({"win_rate": "{:.1%}", "elo": "{:.0f}"})
      .background_gradient(axis=0, subset=["elo"])
)


,player,wins,losses,games,win_rate,elo
0,openai/o4-mini,7,0,7,100.0%,1094
1,google/gemini-2.5-pro,5,3,8,62.5%,1021
2,Randy,8,9,17,47.1%,996
3,openai/o3,4,6,10,40.0%,967
4,anthropic/claude-4-sonnet-20250522,2,8,10,20.0%,921
